In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch

from sbi import utils as utils
from sbi.analysis import pairplot
from sbi.inference import NSPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)

In [ ]:
num_dim = 2
prior = utils.BoxUniform(low=-2 * torch.ones(num_dim), high=2 * torch.ones(num_dim))

def simulator(theta):
    # linear gaussian
    return theta + 1.0 + torch.randn_like(theta) * 0.1

# Check prior, simulator, consistency
prior, num_parameters, prior_returns_numpy = process_prior(prior)
simulator = process_simulator(simulator, prior, prior_returns_numpy)
check_sbi_inputs(simulator, prior)

In [ ]:
# Create inference object. Here, NPE is used.
inference = NSPE(prior=None, sde_type="ve")

# generate simulations and pass to the inference object
theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=5_000)
inference = inference.append_simulations(theta, x)

# train the density estimator and build the posterior
score_estimator = inference.train(stop_after_epochs=1000)
posterior = inference.build_posterior(score_estimator)

In [ ]:
from sbi.analysis import plot_summary

plot_summary(inference, tags=["training_loss", "validation_loss"])

In [ ]:
posterior = inference.build_posterior(score_estimator)

In [ ]:
theta_true = prior.sample((1,))
# generate our observation
x_obs = simulator(theta_true)

In [ ]:
from sbi.samplers.score.predictors import DDIM, EulerMaruyama 


In [ ]:
samples = posterior.sample((10000,), x=x_obs, steps=500)*3

In [ ]:
samples = torch.rand(10000, 2) * 4 - 2

In [ ]:
log_probs = posterior.log_prob(samples, x=x_obs)

In [ ]:
posterior.set_default_x(x_obs)

In [ ]:
map = posterior.map(learning_rate=1e-6)

In [ ]:
samples = posterior.sample((10000,), x=x_obs)
#samples = x.detach()
_ = pairplot(samples, points=theta_true, limits=[[-2, 2], [-2, 2], [-2, 2]], figsize=(6, 6), labels=[r"$\theta_1$", r"$\theta_2$", r"$\theta_3$"])

In [ ]:
from zuko.transforms import FreeFormJacobianTransform

def build_freeform_jacobian_transform(x_o, atol=1e-5, rtol=1e-6, exact=True):
    # Create a freeform jacobian transformation
    phi = score_estimator.parameters()
    def f(t,x):
        score = score_estimator(input=x, condition=x_o, time=t)
        f = score_estimator.drift_fn(x,t)
        g = score_estimator.diffusion_fn(x,t)
        v = f - 0.5*g**2 * score
        return v

    transform = FreeFormJacobianTransform(
        f=f, t0= score_estimator.T_min, t1=score_estimator.T_max, phi=phi, atol=atol, rtol=rtol, exact=exact
    )
    return transform

In [ ]:
x_o=x_obs

def f(t,x):
    t = torch.atleast_1d(t)
    score = score_estimator(input=x, condition=x_o, time=t)
    f = score_estimator.drift_fn(x, t)
    g = score_estimator.diffusion_fn(x,t)


    v = f -  0.5*g**2*score
    return v

In [ ]:
t = build_freeform_jacobian_transform(x_o=x_obs)

In [ ]:
x = t.inv(torch.randn(1000,3)*5)

In [ ]:
t.inv.call_and_ladj(torch.randn(1000,3)*5)

In [ ]:
ts = torch.linspace(1,1e-5,1000)
x0s = torch.randn(100,3)*5.
x = x0s
for i in range(1000):
    x -= f(ts[i],x) * 1e-3
    

In [ ]:
samples = x.detach()


_ = pairplot(samples, points=theta_true, limits=[[-2, 2], [-2, 2], [-2, 2]], figsize=(6, 6), labels=[r"$\theta_1$", r"$\theta_2$", r"$\theta_3$"])

In [ ]:
!pip install sbibm --no-deps

In [ ]:
from sbibm.tasks import get_task

task = get_task("slcp")

prior = task.get_prior_dist()
simulator = task.get_simulator()

thetas = prior.sample((100_000,))
xs = simulator(thetas)


In [ ]:
# Create inference object. Here, NPE is used.
inference = NSPE(prior=prior, sde_type="subvp")
inference = inference.append_simulations(thetas, xs)

# train the density estimator and build the posterior
score_estimator = inference.train(stop_after_epochs=50, training_batch_size=100)
posterior = inference.build_posterior(score_estimator)

In [ ]:
from sbi.analysis import plot_summary

_ = plot_summary(inference, tags=["training_loss", "validation_loss"])

In [ ]:
score_estimator.std_fn(score_estimator.T_min*torch.ones((1,)))

In [ ]:
from sbi.utils.metrics import c2st 
import matplotlib.pyplot as plt

for i in range(1,2):
    ref_samples = task.get_reference_posterior_samples(i)
    x_obs = task.get_observation(i)
    samples = posterior.sample((10000,), x=x_obs, steps=1000)
    print(c2st(ref_samples, samples))
    plt.scatter(samples[:,0], samples[:,1],s=1)
    plt.scatter(ref_samples[:,0], ref_samples[:,1], s=1)
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.show()

In [ ]:
# This file is part of sbi, a toolkit for simulation-based inference. sbi is licensed
# under the Apache License Version 2.0, see <https://www.apache.org/licenses/>

from __future__ import annotations

from typing import Tuple

import pytest
import torch
from torch import Tensor

from sbi.neural_nets.score_nets import build_score_estimator
from sbi.inference.potentials.score_based_potential import (
    score_estimator_based_potential_gradient,
)
from sbi.samplers.score.score import Diffuser


@pytest.mark.parametrize(
    "sde_type",
    [
        "vp",
        "ve",
        "subvp",
    ],
)
@pytest.mark.parametrize("input_event_shape", ((1,), (4,)))
@pytest.mark.parametrize("std", (1.0, 0.1))
def test_score_estimator_forward_shapes(sde_type, input_event_shape, std):

    mean0 = torch.zeros(input_event_shape)
    std0 = std * torch.ones(input_event_shape)

    score_fn = _build_gaussian_score_estimator(sde_type, input_event_shape, mean0, std0)

    sampler = Diffuser(score_fn, "euler_maruyama", None)

    T_min = score_fn.score_estimator.T_min
    T_max = score_fn.score_estimator.T_max
    ts = torch.linspace(T_max, T_min, 1000)
    samples = sampler.run(10_000, ts)

    mean_est = samples[0].mean(0)
    std_est = samples[0].std(0)

    # TODO: Fix this

    # print(mean_est, std_est)
    # assert torch.allclose(mean_est, torch.zeros_like(mean_est), rtol=1e-3)
    # assert torch.allclose(std_est, torch.ones_like(mean_est) * std, rtol=1e-3)



def _build_gaussian_score_estimator(
    sde_type: str,
    input_event_shape: Tuple[int],
    mean0: Tensor,
    std0: Tensor,
):
    """Helper function for all tests that deal with shapes of density estimators."""

    # Use discrete thetas such that categorical density esitmators can also use them.
    building_thetas = (
        torch.randn((1000, *input_event_shape), dtype=torch.float32) * std0
        + mean0
    )
    building_xs = torch.ones((1000, 1))

    class DummyNet(torch.nn.Module):
        def forward(self, x):
            return torch.zeros((x.shape[0], *input_event_shape))
    

    score_estimator = build_score_estimator(
        building_thetas,
        building_xs,
        sde_type=sde_type,
        score_net=DummyNet(),
    )
    

    score_fn, _ = score_estimator_based_potential_gradient(
        score_estimator, prior=None, x_o=torch.ones((1,))
    )

    return score_fn


In [ ]:
input_event_shape = (1,)
std = 1.
sde_type="ve"

mean0 = torch.zeros(input_event_shape)
std0 = std * torch.ones(input_event_shape)

score_fn = _build_gaussian_score_estimator(sde_type, input_event_shape, mean0, std0)

sampler = Diffuser(score_fn, "ddim", None)

T_min = score_fn.score_estimator.T_min
T_max = score_fn.score_estimator.T_max
ts = torch.linspace(T_max, T_min, 500)
samples = sampler.run(10_000, ts)

In [ ]:
mean_est = samples[0].mean(0)
std_est = samples[0].std(0)

print(mean_est, std_est)

In [ ]:
mean_est

In [ ]:
score_fn(torch.ones((1,)),1e-3* torch.ones((1,)))

In [ ]:
plt.hist(samples.flatten().numpy())

In [ ]:
from sbi import utils
from sbi.inference import SNPE, infer

import torch

# Example is taken from 00_getting_started.ipynb
num_dim = 3
prior = utils.BoxUniform(low=-2 * torch.ones(num_dim), high=2 * torch.ones(num_dim))

def simulator(parameter_set):
    return 1.0 + parameter_set + torch.randn(parameter_set.shape) * 0.1

posterior = infer(simulator, prior, method="SNPE_A", num_simulations=10)
assert posterior is not None, "Most basic use of 'infer' failed"
posterior = infer(
    simulator,
    prior,
    method="SNPE_A",
    num_simulations=10,
    init_kwargs={"num_components": 5},
    train_kwargs={"max_num_epochs": 2},
    build_posterior_kwargs={"prior": prior},
)
assert posterior is not None, "Using 'infer' with keyword arguments failed"

In [ ]:
import pytest
from pyro.infer.mcmc import MCMC
from torch import Tensor, eye, zeros
from torch.distributions import MultivariateNormal

from sbi.inference import (
    SNL,
    MCMCPosterior,
    likelihood_estimator_based_potential,
    simulate_for_sbi,
)
from sbi.samplers.mcmc import PyMCSampler, SliceSamplerSerial, SliceSamplerVectorized
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
from sbi.utils.user_input_checks import process_prior, process_simulator


In [ ]:

sampling_method: str = "slice_np_vectorized"
num_chains: int = 4,
mcmc_params_fast: dict = {}
num_dim: int = 2
num_samples: int = 42
num_trials: int = 2
num_simulations: int = 10

x_o = zeros((num_trials, num_dim))
mcmc_params_fast["num_chains"] = num_chains

prior = MultivariateNormal(loc=zeros(num_dim), covariance_matrix=eye(num_dim))
simulator = diagonal_linear_gaussian

inference = SNL(prior, show_progress_bars=False)

prior, _, prior_returns_numpy = process_prior(prior)
simulator = process_simulator(simulator, prior, prior_returns_numpy)
theta, x = simulate_for_sbi(
    simulator, prior, num_simulations, simulation_batch_size=10
)
estimator = inference.append_simulations(theta, x).train(max_num_epochs=5)
potential_fn, transform = likelihood_estimator_based_potential(
    estimator, prior, x_o
)
posterior = MCMCPosterior(
    potential_fn, theta_transform=transform, method=sampling_method, proposal=prior
)

assert posterior.posterior_sampler is None
samples = posterior.sample(
    sample_shape=(num_samples, num_chains),
    x=x_o,
    mcmc_parameters={"init_strategy": "prior", **mcmc_params_fast},
)
# assert isinstance(samples, Tensor)
# assert samples.shape == (num_samples, num_chains, num_dim)

# if "pyro" in sampling_method:
#     assert type(posterior.posterior_sampler) is MCMC
# elif "pymc" in sampling_method:
#     assert type(posterior.posterior_sampler) is PyMCSampler
# elif sampling_method == "slice_np":
#     assert type(posterior.posterior_sampler) is SliceSamplerSerial
# else:  # sampling_method == "slice_np_vectorized"
#     assert type(posterior.posterior_sampler) is SliceSamplerVectorized


In [ ]:
posterior.default_x.shape